In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import os

# ==========================================
# CONFIGURATION
# ==========================================
FUT_FILE = "NIFTY20NOV.csv"                 # futures tick file
TRADES_FILE = "kinetic_full_backtest_trades.csv" # kinetic backtest output with Entry/Exit
OPTION_PREFIX = "NIFTY25NOV"                # prefix for options files
STRIKES = [26100, 26150, 26200, 26250, 26300]  # strikes you have
LOT_SIZE = 75                               # NIFTY lot
HEDGE_STEP_THRESHOLD = 0.02                 # min delta change to rebalance (in futures)
HEDGE_COST_PER_CONTRACT = 0.2               # points per futures contract per hedge (slippage+fees)
EXPIRY_DATE = pd.Timestamp("2025-11-27")    # <-- SET CORRECT EXPIRY HERE

print("\n==============================")
print("DELTA-HEDGED GAMMA SCALPING")
print("==============================\n")

# ==========================================
# HELPER: NORMAL PDF & CDF
# ==========================================
def norm_pdf(x):
    return 1.0 / math.sqrt(2 * math.pi) * math.exp(-0.5 * x * x)

def norm_cdf(x):
    # Using error function approximation
    return 0.5 * (1 + math.erf(x / math.sqrt(2)))


# ==========================================
# BLACK-SCHOLES: CALL PRICE & DELTA
# ==========================================
def bs_call_price(S, K, T, sigma, r=0.0):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return max(S - K, 0.0)
    sqrtT = math.sqrt(T)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT
    return S * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)

def bs_call_delta(S, K, T, sigma, r=0.0):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return 1.0 if S > K else 0.0
    sqrtT = math.sqrt(T)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrtT)
    return norm_cdf(d1)


# ==========================================
# IMPLIED VOL (BISECTION)
# ==========================================
def implied_vol_call_bisect(market_price, S, K, T, r=0.0, 
                            sigma_low=0.0001, sigma_high=3.0, 
                            tol=1e-4, max_iter=50):
    """Simple bisection implied vol for call. Returns sigma or None."""
    if T <= 0:
        return None

    # Check bounds
    price_low = bs_call_price(S, K, T, sigma_low, r)
    price_high = bs_call_price(S, K, T, sigma_high, r)
    if price_low > market_price or price_high < market_price:
        # Outside theoretical range, fallback
        return None

    for _ in range(max_iter):
        sigma_mid = 0.5 * (sigma_low + sigma_high)
        price_mid = bs_call_price(S, K, T, sigma_mid, r)

        if abs(price_mid - market_price) < tol:
            return sigma_mid

        if price_mid > market_price:
            sigma_high = sigma_mid
        else:
            sigma_low = sigma_mid

    return sigma_mid


# ==========================================
# LOAD FUTURES DATA (NIFTY20NOV.csv)
# ==========================================
print("Loading futures data...")
fut_df = pd.read_csv(FUT_FILE)

# Expecting columns: 'Date', 'Time', 'LTP'
# Adjust if your columns have different names
fut_df['DateTime'] = pd.to_datetime(
    fut_df['Date'] + ' ' + fut_df['Time'],
    format='%d/%m/%Y %H:%M:%S.%f',
    errors='coerce'
)
fut_df = fut_df.dropna(subset=['DateTime'])
fut_df['LTP'] = pd.to_numeric(fut_df['LTP'], errors='coerce')
fut_df = fut_df.dropna(subset=['LTP'])
fut_df = fut_df.sort_values('DateTime').reset_index(drop=True)

print(f"Futures ticks: {len(fut_df):,}")
print(f"From {fut_df['DateTime'].min()} to {fut_df['DateTime'].max()}\n")


# ==========================================
# LOAD KINETIC TRADES (ENTRY/EXIT WINDOWS)
# ==========================================
print("Loading kinetic trade windows...")
trades_raw = pd.read_csv(TRADES_FILE)

# Expecting 'Entry_Time' and 'Exit_Time'
trades_raw['Entry_Time'] = pd.to_datetime(trades_raw['Entry_Time'])
trades_raw['Exit_Time']  = pd.to_datetime(trades_raw['Exit_Time'])

# Filter only trades for this specific day (20 Nov) if needed:
# trades_raw = trades_raw[trades_raw['Entry_Time'].dt.date == datetime.date(2025, 11, 20)]

trades_raw = trades_raw.sort_values('Entry_Time').reset_index(drop=True)
print(f"Number of kinetic trades in file: {len(trades_raw)}\n")


# ==========================================
# LOAD OPTION FILES INTO A SINGLE DATAFRAME
# ==========================================
print("Loading option data (CE + PE for strikes)...")
opt_frames = []

for K in STRIKES:
    for opt_type in ['CE', 'PE']:
        fname = f"{OPTION_PREFIX}{K}{opt_type}.parquet"
        if not os.path.exists(fname):
            print(f"WARNING: Missing file {fname}, skipping.")
            continue
        
        df_opt = pd.read_parquet(fname)
        # Expecting 'Date', 'Time', 'LTP' similarly
        df_opt['DateTime'] = pd.to_datetime(
            df_opt['Date'] + ' ' + df_opt['Time'],
            format='%d/%m/%Y %H:%M:%S.%f',
            errors='coerce'
        )
        df_opt = df_opt.dropna(subset=['DateTime'])
        df_opt['LTP'] = pd.to_numeric(df_opt['LTP'], errors='coerce')
        df_opt = df_opt.dropna(subset=['LTP'])

        df_opt['Strike'] = float(K)
        df_opt['Option_Type'] = opt_type  # 'CE' or 'PE'
        opt_frames.append(df_opt[['DateTime', 'LTP', 'Strike', 'Option_Type']])

opt_df = pd.concat(opt_frames, ignore_index=True).sort_values('DateTime')
print(f"Total option ticks: {len(opt_df):,}\n")


# ==========================================
# HELPER: GET SPOT AT TIME (ASOF)
# ==========================================
fut_df = fut_df.sort_values('DateTime').reset_index(drop=True)
fut_df.set_index('DateTime', inplace=True)

opt_df = opt_df.sort_values('DateTime').reset_index(drop=True)
opt_df.set_index('DateTime', inplace=True)


def get_spot_at(ts):
    """Get futures LTP at or just before ts."""
    if ts < fut_df.index[0]:
        return None
    loc = fut_df.index.searchsorted(ts, side='right') - 1
    if loc < 0:
        return None
    return fut_df.iloc[loc]['LTP']


def slice_ts(df, start_ts, end_ts):
    """Slice df between start_ts and end_ts inclusive."""
    return df.loc[(df.index >= start_ts) & (df.index <= end_ts)].copy()


# ==========================================
# GAMMA SCALPING SIM FOR ONE TRADE WINDOW
# ==========================================
def gamma_scalp_for_trade(row):
    entry_ts = row['Entry_Time']
    exit_ts  = row['Exit_Time']

    # 1) Get spot at entry
    S0 = get_spot_at(entry_ts)
    if S0 is None:
        return None

    # 2) Pick ATM strike from list (closest to S0)
    atm_strike = min(STRIKES, key=lambda K: abs(K - S0))

    # 3) Extract CE & PE series for that strike in [entry, exit]
    ce_series = slice_ts(
        opt_df[(opt_df['Strike'] == atm_strike) & (opt_df['Option_Type'] == 'CE')],
        entry_ts, exit_ts
    )
    pe_series = slice_ts(
        opt_df[(opt_df['Strike'] == atm_strike) & (opt_df['Option_Type'] == 'PE')],
        entry_ts, exit_ts
    )

    # Require some data
    if ce_series.empty or pe_series.empty:
        return None

    # 4) Build 1-second grid
    time_index = pd.date_range(start=entry_ts, end=exit_ts, freq='1S')

    # Reindex futures & options to this grid
    fut_sub = slice_ts(fut_df, entry_ts, exit_ts)
    if fut_sub.empty:
        return None

    S_t = fut_sub['LTP'].reindex(time_index, method='ffill')
    CE_t = ce_series['LTP'].reindex(time_index, method='ffill')
    PE_t = pe_series['LTP'].reindex(time_index, method='ffill')

    # Drop any times where we don't have all
    valid = S_t.notna() & CE_t.notna() & PE_t.notna()
    S_t = S_t[valid]
    CE_t = CE_t[valid]
    PE_t = PE_t[valid]
    time_index = S_t.index

    if len(time_index) < 5:
        return None

    # 5) ENTRY
    S_entry = S_t.iloc[0]
    CE_entry = CE_t.iloc[0]
    PE_entry = PE_t.iloc[0]

    # Time to expiry at entry
    T0 = (EXPIRY_DATE - time_index[0]).total_seconds() / (365.0 * 24 * 3600)
    T0 = max(T0, 1e-6)

    K = atm_strike

    # Implied vol for CE at entry
    sigma_ce = implied_vol_call_bisect(CE_entry, S_entry, K, T0)
    # For PE, we can use put-call parity to infer call-equivalent price:
    # C = P + S - K e^{-rT}, so treat that as "call price" for iv
    ce_equiv_from_put = PE_entry + S_entry - K  # r≈0
    sigma_pe = implied_vol_call_bisect(max(ce_equiv_from_put, 0.0001), S_entry, K, T0)

    if sigma_ce is None or sigma_pe is None:
        # fallback: use some rough vol (e.g. 0.15)
        sigma_ce = sigma_ce or 0.15
        sigma_pe = sigma_pe or 0.15

    # We are long 1 CE and 1 PE
    option_cost = CE_entry + PE_entry

    # 6) Start delta hedge
    hedge_pos = 0.0      # futures contracts
    hedge_cash_pnl = 0.0 # cash from futures trades
    hedge_trades = 0

    # 7) Time loop
    for t in time_index:
        S = S_t.loc[t]
        CE = CE_t.loc[t]
        PE = PE_t.loc[t]

        # Time to expiry at time t
        T_t = (EXPIRY_DATE - t).total_seconds() / (365.0 * 24 * 3600)
        T_t = max(T_t, 1e-6)

        # Call delta using CE's implied vol (gamma mostly from CE for ATM)
        delta_call = bs_call_delta(S, K, T_t, sigma_ce)
        # Put delta from call delta via parity: Δ_put = Δ_call - 1
        delta_put = delta_call - 1.0

        opt_delta = delta_call + delta_put  # total delta of straddle

        # Desired futures position to be net delta ~ 0
        desired_hedge_pos = -opt_delta  # 1 future = 1 delta per point

        # Adjust only if significant change
        delta_change = desired_hedge_pos - hedge_pos
        if abs(delta_change) > HEDGE_STEP_THRESHOLD:
            # Trade this much futures at price S
            # Negative delta_change => we buy futures (spend cash)
            hedge_cash_pnl -= delta_change * S
            hedge_pos = desired_hedge_pos
            hedge_trades += abs(delta_change)

    # 8) At EXIT: mark to market
    S_exit = S_t.iloc[-1]
    CE_exit = CE_t.iloc[-1]
    PE_exit = PE_t.iloc[-1]

    option_pnl = (CE_exit + PE_exit) - option_cost
    futures_pnl = hedge_pos * S_exit + hedge_cash_pnl

    gross_pnl_points = option_pnl + futures_pnl

    # Hedge costs: per-contract per trade in points
    # Treat hedge_trades as "futures contracts traded" approx
    hedge_cost_points = hedge_trades * HEDGE_COST_PER_CONTRACT

    net_pnl_points = gross_pnl_points - hedge_cost_points
    net_pnl_rupees = net_pnl_points * LOT_SIZE

    return {
        "Date": entry_ts.date(),
        "Entry_Time": entry_ts,
        "Exit_Time": exit_ts,
        "ATM_Strike": atm_strike,
        "S_Entry": S_entry,
        "S_Exit": S_exit,
        "CE_Entry": CE_entry,
        "CE_Exit": CE_exit,
        "PE_Entry": PE_entry,
        "PE_Exit": PE_exit,
        "Sigma_CE": sigma_ce,
        "Sigma_PE": sigma_pe,
        "Option_PnL_pts": option_pnl,
        "Futures_PnL_pts": futures_pnl,
        "Gross_PnL_pts": gross_pnl_points,
        "Hedge_Trades": hedge_trades,
        "Hedge_Cost_pts": hedge_cost_points,
        "Net_PnL_pts": net_pnl_points,
        "Net_PnL_rupees": net_pnl_rupees
    }


# ==========================================
# RUN OVER ALL TRADES
# ==========================================
results = []
for i, row in trades_raw.iterrows():
    res = gamma_scalp_for_trade(row)
    if res is not None:
        results.append(res)
    else:
        print(f"Skipped trade {i} (insufficient data or mismatch).")

if not results:
    print("\nNo valid gamma-scalping simulations completed.")
else:
    res_df = pd.DataFrame(results)
    print("\n========= GAMMA SCALPING SUMMARY =========")
    print(res_df[['Date', 'Entry_Time', 'Exit_Time', 'ATM_Strike',
                  'Net_PnL_pts', 'Net_PnL_rupees']].head())

    total_pts = res_df['Net_PnL_pts'].sum()
    total_rs  = res_df['Net_PnL_rupees'].sum()
    print("\nTOTAL Net Gamma PnL:")
    print(f"  {total_pts:.2f} points")
    print(f"  ₹{total_rs:,.2f}")

    print("\nPer-trade stats:")
    print(f"  Trades: {len(res_df)}")
    print(f"  Avg per trade: {res_df['Net_PnL_pts'].mean():.2f} pts "
          f"(₹{res_df['Net_PnL_rupees'].mean():.2f})")
    print(f"  Win rate: "
          f"{(res_df['Net_PnL_pts'] > 0).mean() * 100:.1f}%")

    # Save to CSV
    out_file = "gamma_scalping_results.csv"
    res_df.to_csv(out_file, index=False)
    print(f"\nDetailed gamma scalping results saved to: {out_file}")



DELTA-HEDGED GAMMA SCALPING

Loading futures data...


FileNotFoundError: [Errno 2] No such file or directory: 'NIFTY20NOV.csv'

In [ ]:
import pandas as pd
import numpy as np
import datetime
import math
import os

# ==========================================
# CONFIGURATION
# ==========================================
FUT_FILE = "NIFTY20NOV.csv"                 # futures tick file
TRADES_FILE = "kinetic_full_backtest_trades.csv" # kinetic backtest output with Entry/Exit
OPTION_PREFIX = "NIFTY25NOV"                # prefix for options files
STRIKES = [26100, 26150, 26200, 26250, 26300]  # strikes you have
LOT_SIZE = 75                               # NIFTY lot
HEDGE_STEP_THRESHOLD = 0.02                 # min delta change to rebalance (in futures)
HEDGE_COST_PER_CONTRACT = 0.2               # points per futures contract per hedge (slippage+fees)
EXPIRY_DATE = pd.Timestamp("2025-11-27")    # <-- SET CORRECT EXPIRY HERE

print("\n==============================")
print("DELTA-HEDGED GAMMA SCALPING")
print("==============================\n")

# ==========================================
# HELPER: NORMAL PDF & CDF
# ==========================================
def norm_pdf(x):
    return 1.0 / math.sqrt(2 * math.pi) * math.exp(-0.5 * x * x)

def norm_cdf(x):
    # Using error function approximation
    return 0.5 * (1 + math.erf(x / math.sqrt(2)))


# ==========================================
# BLACK-SCHOLES: CALL PRICE & DELTA
# ==========================================
def bs_call_price(S, K, T, sigma, r=0.0):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return max(S - K, 0.0)
    sqrtT = math.sqrt(T)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT
    return S * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)

def bs_call_delta(S, K, T, sigma, r=0.0):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return 1.0 if S > K else 0.0
    sqrtT = math.sqrt(T)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrtT)
    return norm_cdf(d1)


# ==========================================
# IMPLIED VOL (BISECTION)
# ==========================================
def implied_vol_call_bisect(market_price, S, K, T, r=0.0, 
                            sigma_low=0.0001, sigma_high=3.0, 
                            tol=1e-4, max_iter=50):
    """Simple bisection implied vol for call. Returns sigma or None."""
    if T <= 0:
        return None

    # Check bounds
    price_low = bs_call_price(S, K, T, sigma_low, r)
    price_high = bs_call_price(S, K, T, sigma_high, r)
    if price_low > market_price or price_high < market_price:
        # Outside theoretical range, fallback
        return None

    for _ in range(max_iter):
        sigma_mid = 0.5 * (sigma_low + sigma_high)
        price_mid = bs_call_price(S, K, T, sigma_mid, r)

        if abs(price_mid - market_price) < tol:
            return sigma_mid

        if price_mid > market_price:
            sigma_high = sigma_mid
        else:
            sigma_low = sigma_mid

    return sigma_mid


# ==========================================
# LOAD FUTURES DATA (NIFTY20NOV.csv)
# ==========================================
print("Loading futures data...")
fut_df = pd.read_csv(FUT_FILE)

# Expecting columns: 'Date', 'Time', 'LTP'
# Adjust if your columns have different names
fut_df['DateTime'] = pd.to_datetime(
    fut_df['Date'] + ' ' + fut_df['Time'],
    format='%d/%m/%Y %H:%M:%S.%f',
    errors='coerce'
)
fut_df = fut_df.dropna(subset=['DateTime'])
fut_df['LTP'] = pd.to_numeric(fut_df['LTP'], errors='coerce')
fut_df = fut_df.dropna(subset=['LTP'])
fut_df = fut_df.sort_values('DateTime').reset_index(drop=True)

print(f"Futures ticks: {len(fut_df):,}")
print(f"From {fut_df['DateTime'].min()} to {fut_df['DateTime'].max()}\n")


# ==========================================
# LOAD KINETIC TRADES (ENTRY/EXIT WINDOWS)
# ==========================================
print("Loading kinetic trade windows...")
trades_raw = pd.read_csv(TRADES_FILE)

# Expecting 'Entry_Time' and 'Exit_Time'
trades_raw['Entry_Time'] = pd.to_datetime(trades_raw['Entry_Time'])
trades_raw['Exit_Time']  = pd.to_datetime(trades_raw['Exit_Time'])

# Filter only trades for this specific day (20 Nov) if needed:
# trades_raw = trades_raw[trades_raw['Entry_Time'].dt.date == datetime.date(2025, 11, 20)]

trades_raw = trades_raw.sort_values('Entry_Time').reset_index(drop=True)
print(f"Number of kinetic trades in file: {len(trades_raw)}\n")


# ==========================================
# LOAD OPTION FILES INTO A SINGLE DATAFRAME
# ==========================================
print("Loading option data (CE + PE for strikes)...")
opt_frames = []

for K in STRIKES:
    for opt_type in ['CE', 'PE']:
        fname = f"{OPTION_PREFIX}{K}{opt_type}.parquet"
        if not os.path.exists(fname):
            print(f"WARNING: Missing file {fname}, skipping.")
            continue
        
        df_opt = pd.read_parquet(fname)
        # Expecting 'Date', 'Time', 'LTP' similarly
        df_opt['DateTime'] = pd.to_datetime(
            df_opt['Date'] + ' ' + df_opt['Time'],
            format='%d/%m/%Y %H:%M:%S.%f',
            errors='coerce'
        )
        df_opt = df_opt.dropna(subset=['DateTime'])
        df_opt['LTP'] = pd.to_numeric(df_opt['LTP'], errors='coerce')
        df_opt = df_opt.dropna(subset=['LTP'])

        df_opt['Strike'] = float(K)
        df_opt['Option_Type'] = opt_type  # 'CE' or 'PE'
        opt_frames.append(df_opt[['DateTime', 'LTP', 'Strike', 'Option_Type']])

opt_df = pd.concat(opt_frames, ignore_index=True).sort_values('DateTime')
print(f"Total option ticks: {len(opt_df):,}\n")


# ==========================================
# HELPER: GET SPOT AT TIME (ASOF)
# ==========================================
fut_df = fut_df.sort_values('DateTime').reset_index(drop=True)
fut_df.set_index('DateTime', inplace=True)

opt_df = opt_df.sort_values('DateTime').reset_index(drop=True)
opt_df.set_index('DateTime', inplace=True)


def get_spot_at(ts):
    """Get futures LTP at or just before ts."""
    if ts < fut_df.index[0]:
        return None
    loc = fut_df.index.searchsorted(ts, side='right') - 1
    if loc < 0:
        return None
    return fut_df.iloc[loc]['LTP']


def slice_ts(df, start_ts, end_ts):
    """Slice df between start_ts and end_ts inclusive."""
    return df.loc[(df.index >= start_ts) & (df.index <= end_ts)].copy()


# ==========================================
# GAMMA SCALPING SIM FOR ONE TRADE WINDOW
# ==========================================
def gamma_scalp_for_trade(row):
    entry_ts = row['Entry_Time']
    exit_ts  = row['Exit_Time']

    # 1) Get spot at entry
    S0 = get_spot_at(entry_ts)
    if S0 is None:
        return None

    # 2) Pick ATM strike from list (closest to S0)
    atm_strike = min(STRIKES, key=lambda K: abs(K - S0))

    # 3) Extract CE & PE series for that strike in [entry, exit]
    ce_series = slice_ts(
        opt_df[(opt_df['Strike'] == atm_strike) & (opt_df['Option_Type'] == 'CE')],
        entry_ts, exit_ts
    )
    pe_series = slice_ts(
        opt_df[(opt_df['Strike'] == atm_strike) & (opt_df['Option_Type'] == 'PE')],
        entry_ts, exit_ts
    )

    # Require some data
    if ce_series.empty or pe_series.empty:
        return None

    # 4) Build 1-second grid
    time_index = pd.date_range(start=entry_ts, end=exit_ts, freq='1S')

    # Reindex futures & options to this grid
    fut_sub = slice_ts(fut_df, entry_ts, exit_ts)
    if fut_sub.empty:
        return None

    S_t = fut_sub['LTP'].reindex(time_index, method='ffill')
    CE_t = ce_series['LTP'].reindex(time_index, method='ffill')
    PE_t = pe_series['LTP'].reindex(time_index, method='ffill')

    # Drop any times where we don't have all
    valid = S_t.notna() & CE_t.notna() & PE_t.notna()
    S_t = S_t[valid]
    CE_t = CE_t[valid]
    PE_t = PE_t[valid]
    time_index = S_t.index

    if len(time_index) < 5:
        return None

    # 5) ENTRY
    S_entry = S_t.iloc[0]
    CE_entry = CE_t.iloc[0]
    PE_entry = PE_t.iloc[0]

    # Time to expiry at entry
    T0 = (EXPIRY_DATE - time_index[0]).total_seconds() / (365.0 * 24 * 3600)
    T0 = max(T0, 1e-6)

    K = atm_strike

    # Implied vol for CE at entry
    sigma_ce = implied_vol_call_bisect(CE_entry, S_entry, K, T0)
    # For PE, we can use put-call parity to infer call-equivalent price:
    # C = P + S - K e^{-rT}, so treat that as "call price" for iv
    ce_equiv_from_put = PE_entry + S_entry - K  # r≈0
    sigma_pe = implied_vol_call_bisect(max(ce_equiv_from_put, 0.0001), S_entry, K, T0)

    if sigma_ce is None or sigma_pe is None:
        # fallback: use some rough vol (e.g. 0.15)
        sigma_ce = sigma_ce or 0.15
        sigma_pe = sigma_pe or 0.15

    # We are long 1 CE and 1 PE
    option_cost = CE_entry + PE_entry

    # 6) Start delta hedge
    hedge_pos = 0.0      # futures contracts
    hedge_cash_pnl = 0.0 # cash from futures trades
    hedge_trades = 0

    # 7) Time loop
    for t in time_index:
        S = S_t.loc[t]
        CE = CE_t.loc[t]
        PE = PE_t.loc[t]

        # Time to expiry at time t
        T_t = (EXPIRY_DATE - t).total_seconds() / (365.0 * 24 * 3600)
        T_t = max(T_t, 1e-6)

        # Call delta using CE's implied vol (gamma mostly from CE for ATM)
        delta_call = bs_call_delta(S, K, T_t, sigma_ce)
        # Put delta from call delta via parity: Δ_put = Δ_call - 1
        delta_put = delta_call - 1.0

        opt_delta = delta_call + delta_put  # total delta of straddle

        # Desired futures position to be net delta ~ 0
        desired_hedge_pos = -opt_delta  # 1 future = 1 delta per point

        # Adjust only if significant change
        delta_change = desired_hedge_pos - hedge_pos
        if abs(delta_change) > HEDGE_STEP_THRESHOLD:
            # Trade this much futures at price S
            # Negative delta_change => we buy futures (spend cash)
            hedge_cash_pnl -= delta_change * S
            hedge_pos = desired_hedge_pos
            hedge_trades += abs(delta_change)

    # 8) At EXIT: mark to market
    S_exit = S_t.iloc[-1]
    CE_exit = CE_t.iloc[-1]
    PE_exit = PE_t.iloc[-1]

    option_pnl = (CE_exit + PE_exit) - option_cost
    futures_pnl = hedge_pos * S_exit + hedge_cash_pnl

    gross_pnl_points = option_pnl + futures_pnl

    # Hedge costs: per-contract per trade in points
    # Treat hedge_trades as "futures contracts traded" approx
    hedge_cost_points = hedge_trades * HEDGE_COST_PER_CONTRACT

    net_pnl_points = gross_pnl_points - hedge_cost_points
    net_pnl_rupees = net_pnl_points * LOT_SIZE

    return {
        "Date": entry_ts.date(),
        "Entry_Time": entry_ts,
        "Exit_Time": exit_ts,
        "ATM_Strike": atm_strike,
        "S_Entry": S_entry,
        "S_Exit": S_exit,
        "CE_Entry": CE_entry,
        "CE_Exit": CE_exit,
        "PE_Entry": PE_entry,
        "PE_Exit": PE_exit,
        "Sigma_CE": sigma_ce,
        "Sigma_PE": sigma_pe,
        "Option_PnL_pts": option_pnl,
        "Futures_PnL_pts": futures_pnl,
        "Gross_PnL_pts": gross_pnl_points,
        "Hedge_Trades": hedge_trades,
        "Hedge_Cost_pts": hedge_cost_points,
        "Net_PnL_pts": net_pnl_points,
        "Net_PnL_rupees": net_pnl_rupees
    }


# ==========================================
# RUN OVER ALL TRADES
# ==========================================
results = []
for i, row in trades_raw.iterrows():
    res = gamma_scalp_for_trade(row)
    if res is not None:
        results.append(res)
    else:
        print(f"Skipped trade {i} (insufficient data or mismatch).")

if not results:
    print("\nNo valid gamma-scalping simulations completed.")
else:
    res_df = pd.DataFrame(results)
    print("\n========= GAMMA SCALPING SUMMARY =========")
    print(res_df[['Date', 'Entry_Time', 'Exit_Time', 'ATM_Strike',
                  'Net_PnL_pts', 'Net_PnL_rupees']].head())

    total_pts = res_df['Net_PnL_pts'].sum()
    total_rs  = res_df['Net_PnL_rupees'].sum()
    print("\nTOTAL Net Gamma PnL:")
    print(f"  {total_pts:.2f} points")
    print(f"  ₹{total_rs:,.2f}")

    print("\nPer-trade stats:")
    print(f"  Trades: {len(res_df)}")
    print(f"  Avg per trade: {res_df['Net_PnL_pts'].mean():.2f} pts "
          f"(₹{res_df['Net_PnL_rupees'].mean():.2f})")
    print(f"  Win rate: "
          f"{(res_df['Net_PnL_pts'] > 0).mean() * 100:.1f}%")

    # Save to CSV
    out_file = "gamma_scalping_results.csv"
    res_df.to_csv(out_file, index=False)
    print(f"\nDetailed gamma scalping results saved to: {out_file}")


In [2]:
import boto3
import pandas as pd
from io import BytesIO

BUCKET = "live-market-data"
YEAR = 2025
MONTH = 11
DAY = 4
SYMBOL = "NIFTY"

def get_fut_from_s3(prefix):
    s3 = boto3.client("s3")
    resp = s3.list_objects_v2(Bucket=BUCKET, Prefix=prefix)

    for obj in resp.get("Contents", []):
        key = obj["Key"]
        if key.endswith(".parquet"):
            data = s3.get_object(Bucket=BUCKET, Key=key)["Body"].read()
            df = pd.read_parquet(BytesIO(data))
            print(f"\n--- {key} ---\n")
            return df

df = get_fut_from_s3(f"year={YEAR}/month={MONTH:02d}/day={DAY:02d}/Futures/{SYMBOL}/")



--- year=2025/month=11/day=04/Futures/NIFTY/NIFTY25NOVFUT.parquet ---



In [3]:
import pandas as pd
import numpy as np
import datetime
import math
import os

# ==========================================
# CONFIGURATION
# ==========================================
FUT_FILE = get_fut_from_s3(f"year={YEAR}/month={MONTH:02d}/day={DAY:02d}/Futures/{SYMBOL}/")                 # futures tick file
TRADES_FILE = "kinetic_full_backtest_trades.csv" # kinetic backtest output with Entry/Exit
OPTION_PREFIX = "NIFTY25NOV"                # prefix for options files
STRIKES = [26100, 26150, 26200, 26250, 26300]  # strikes you have
LOT_SIZE = 75                               # NIFTY lot
HEDGE_STEP_THRESHOLD = 0.02                 # min delta change to rebalance (in futures)
HEDGE_COST_PER_CONTRACT = 0.2               # points per futures contract per hedge (slippage+fees)
EXPIRY_DATE = pd.Timestamp("2025-11-27")    # <-- SET CORRECT EXPIRY HERE

print("\n==============================")
print("DELTA-HEDGED GAMMA SCALPING")
print("==============================\n")

# ==========================================
# HELPER: NORMAL PDF & CDF
# ==========================================
def norm_pdf(x):
    return 1.0 / math.sqrt(2 * math.pi) * math.exp(-0.5 * x * x)

def norm_cdf(x):
    # Using error function approximation
    return 0.5 * (1 + math.erf(x / math.sqrt(2)))


# ==========================================
# BLACK-SCHOLES: CALL PRICE & DELTA
# ==========================================
def bs_call_price(S, K, T, sigma, r=0.0):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return max(S - K, 0.0)
    sqrtT = math.sqrt(T)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT
    return S * norm_cdf(d1) - K * math.exp(-r * T) * norm_cdf(d2)

def bs_call_delta(S, K, T, sigma, r=0.0):
    if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
        return 1.0 if S > K else 0.0
    sqrtT = math.sqrt(T)
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrtT)
    return norm_cdf(d1)


# ==========================================
# IMPLIED VOL (BISECTION)
# ==========================================
def implied_vol_call_bisect(market_price, S, K, T, r=0.0, 
                            sigma_low=0.0001, sigma_high=3.0, 
                            tol=1e-4, max_iter=50):
    """Simple bisection implied vol for call. Returns sigma or None."""
    if T <= 0:
        return None

    # Check bounds
    price_low = bs_call_price(S, K, T, sigma_low, r)
    price_high = bs_call_price(S, K, T, sigma_high, r)
    if price_low > market_price or price_high < market_price:
        # Outside theoretical range, fallback
        return None

    for _ in range(max_iter):
        sigma_mid = 0.5 * (sigma_low + sigma_high)
        price_mid = bs_call_price(S, K, T, sigma_mid, r)

        if abs(price_mid - market_price) < tol:
            return sigma_mid

        if price_mid > market_price:
            sigma_high = sigma_mid
        else:
            sigma_low = sigma_mid

    return sigma_mid


# ==========================================
# LOAD FUTURES DATA (NIFTY20NOV.csv)
# ==========================================
print("Loading futures data...")
fut_df = FUT_FILE

# Expecting columns: 'Date', 'Time', 'LTP'
# Adjust if your columns have different names
fut_df['DateTime'] = pd.to_datetime(
    fut_df['Date'] + ' ' + fut_df['Time'],
    format='%d/%m/%Y %H:%M:%S.%f',
    errors='coerce'
)
fut_df = fut_df.dropna(subset=['DateTime'])
fut_df['LTP'] = pd.to_numeric(fut_df['LTP'], errors='coerce')
fut_df = fut_df.dropna(subset=['LTP'])
fut_df = fut_df.sort_values('DateTime').reset_index(drop=True)

print(f"Futures ticks: {len(fut_df):,}")
print(f"From {fut_df['DateTime'].min()} to {fut_df['DateTime'].max()}\n")


# ==========================================
# LOAD KINETIC TRADES (ENTRY/EXIT WINDOWS)
# ==========================================
print("Loading kinetic trade windows...")
trades_raw = pd.read_csv(TRADES_FILE)

# Expecting 'Entry_Time' and 'Exit_Time'
trades_raw['Entry_Time'] = pd.to_datetime(trades_raw['Entry_Time'])
trades_raw['Exit_Time']  = pd.to_datetime(trades_raw['Exit_Time'])

# Filter only trades for this specific day (20 Nov) if needed:
# trades_raw = trades_raw[trades_raw['Entry_Time'].dt.date == datetime.date(2025, 11, 20)]

trades_raw = trades_raw.sort_values('Entry_Time').reset_index(drop=True)
print(f"Number of kinetic trades in file: {len(trades_raw)}\n")


# ==========================================
# LOAD OPTION FILES INTO A SINGLE DATAFRAME
# ==========================================
print("Loading option data (CE + PE for strikes)...")
opt_frames = []

for K in STRIKES:
    for opt_type in ['CE', 'PE']:
        fname = f"{OPTION_PREFIX}{K}{opt_type}.parquet"
        if not os.path.exists(fname):
            print(f"WARNING: Missing file {fname}, skipping.")
            continue
        
        df_opt = pd.read_parquet(fname)
        # Expecting 'Date', 'Time', 'LTP' similarly
        df_opt['DateTime'] = pd.to_datetime(
            df_opt['Date'] + ' ' + df_opt['Time'],
            format='%d/%m/%Y %H:%M:%S.%f',
            errors='coerce'
        )
        df_opt = df_opt.dropna(subset=['DateTime'])
        df_opt['LTP'] = pd.to_numeric(df_opt['LTP'], errors='coerce')
        df_opt = df_opt.dropna(subset=['LTP'])

        df_opt['Strike'] = float(K)
        df_opt['Option_Type'] = opt_type  # 'CE' or 'PE'
        opt_frames.append(df_opt[['DateTime', 'LTP', 'Strike', 'Option_Type']])

opt_df = pd.concat(opt_frames, ignore_index=True).sort_values('DateTime')
print(f"Total option ticks: {len(opt_df):,}\n")


# ==========================================
# HELPER: GET SPOT AT TIME (ASOF)
# ==========================================
fut_df = fut_df.sort_values('DateTime').reset_index(drop=True)
fut_df.set_index('DateTime', inplace=True)

opt_df = opt_df.sort_values('DateTime').reset_index(drop=True)
opt_df.set_index('DateTime', inplace=True)


def get_spot_at(ts):
    """Get futures LTP at or just before ts."""
    if ts < fut_df.index[0]:
        return None
    loc = fut_df.index.searchsorted(ts, side='right') - 1
    if loc < 0:
        return None
    return fut_df.iloc[loc]['LTP']


def slice_ts(df, start_ts, end_ts):
    """Slice df between start_ts and end_ts inclusive."""
    return df.loc[(df.index >= start_ts) & (df.index <= end_ts)].copy()


# ==========================================
# GAMMA SCALPING SIM FOR ONE TRADE WINDOW
# ==========================================
def gamma_scalp_for_trade(row):
    entry_ts = row['Entry_Time']
    exit_ts  = row['Exit_Time']

    # 1) Get spot at entry
    S0 = get_spot_at(entry_ts)
    if S0 is None:
        return None

    # 2) Pick ATM strike from list (closest to S0)
    atm_strike = min(STRIKES, key=lambda K: abs(K - S0))

    # 3) Extract CE & PE series for that strike in [entry, exit]
    ce_series = slice_ts(
        opt_df[(opt_df['Strike'] == atm_strike) & (opt_df['Option_Type'] == 'CE')],
        entry_ts, exit_ts
    )
    pe_series = slice_ts(
        opt_df[(opt_df['Strike'] == atm_strike) & (opt_df['Option_Type'] == 'PE')],
        entry_ts, exit_ts
    )

    # Require some data
    if ce_series.empty or pe_series.empty:
        return None

    # 4) Build 1-second grid
    time_index = pd.date_range(start=entry_ts, end=exit_ts, freq='1S')

    # Reindex futures & options to this grid
    fut_sub = slice_ts(fut_df, entry_ts, exit_ts)
    if fut_sub.empty:
        return None

    S_t = fut_sub['LTP'].reindex(time_index, method='ffill')
    CE_t = ce_series['LTP'].reindex(time_index, method='ffill')
    PE_t = pe_series['LTP'].reindex(time_index, method='ffill')

    # Drop any times where we don't have all
    valid = S_t.notna() & CE_t.notna() & PE_t.notna()
    S_t = S_t[valid]
    CE_t = CE_t[valid]
    PE_t = PE_t[valid]
    time_index = S_t.index

    if len(time_index) < 5:
        return None

    # 5) ENTRY
    S_entry = S_t.iloc[0]
    CE_entry = CE_t.iloc[0]
    PE_entry = PE_t.iloc[0]

    # Time to expiry at entry
    T0 = (EXPIRY_DATE - time_index[0]).total_seconds() / (365.0 * 24 * 3600)
    T0 = max(T0, 1e-6)

    K = atm_strike

    # Implied vol for CE at entry
    sigma_ce = implied_vol_call_bisect(CE_entry, S_entry, K, T0)
    # For PE, we can use put-call parity to infer call-equivalent price:
    # C = P + S - K e^{-rT}, so treat that as "call price" for iv
    ce_equiv_from_put = PE_entry + S_entry - K  # r≈0
    sigma_pe = implied_vol_call_bisect(max(ce_equiv_from_put, 0.0001), S_entry, K, T0)

    if sigma_ce is None or sigma_pe is None:
        # fallback: use some rough vol (e.g. 0.15)
        sigma_ce = sigma_ce or 0.15
        sigma_pe = sigma_pe or 0.15

    # We are long 1 CE and 1 PE
    option_cost = CE_entry + PE_entry

    # 6) Start delta hedge
    hedge_pos = 0.0      # futures contracts
    hedge_cash_pnl = 0.0 # cash from futures trades
    hedge_trades = 0

    # 7) Time loop
    for t in time_index:
        S = S_t.loc[t]
        CE = CE_t.loc[t]
        PE = PE_t.loc[t]

        # Time to expiry at time t
        T_t = (EXPIRY_DATE - t).total_seconds() / (365.0 * 24 * 3600)
        T_t = max(T_t, 1e-6)

        # Call delta using CE's implied vol (gamma mostly from CE for ATM)
        delta_call = bs_call_delta(S, K, T_t, sigma_ce)
        # Put delta from call delta via parity: Δ_put = Δ_call - 1
        delta_put = delta_call - 1.0

        opt_delta = delta_call + delta_put  # total delta of straddle

        # Desired futures position to be net delta ~ 0
        desired_hedge_pos = -opt_delta  # 1 future = 1 delta per point

        # Adjust only if significant change
        delta_change = desired_hedge_pos - hedge_pos
        if abs(delta_change) > HEDGE_STEP_THRESHOLD:
            # Trade this much futures at price S
            # Negative delta_change => we buy futures (spend cash)
            hedge_cash_pnl -= delta_change * S
            hedge_pos = desired_hedge_pos
            hedge_trades += abs(delta_change)

    # 8) At EXIT: mark to market
    S_exit = S_t.iloc[-1]
    CE_exit = CE_t.iloc[-1]
    PE_exit = PE_t.iloc[-1]

    option_pnl = (CE_exit + PE_exit) - option_cost
    futures_pnl = hedge_pos * S_exit + hedge_cash_pnl

    gross_pnl_points = option_pnl + futures_pnl

    # Hedge costs: per-contract per trade in points
    # Treat hedge_trades as "futures contracts traded" approx
    hedge_cost_points = hedge_trades * HEDGE_COST_PER_CONTRACT

    net_pnl_points = gross_pnl_points - hedge_cost_points
    net_pnl_rupees = net_pnl_points * LOT_SIZE

    return {
        "Date": entry_ts.date(),
        "Entry_Time": entry_ts,
        "Exit_Time": exit_ts,
        "ATM_Strike": atm_strike,
        "S_Entry": S_entry,
        "S_Exit": S_exit,
        "CE_Entry": CE_entry,
        "CE_Exit": CE_exit,
        "PE_Entry": PE_entry,
        "PE_Exit": PE_exit,
        "Sigma_CE": sigma_ce,
        "Sigma_PE": sigma_pe,
        "Option_PnL_pts": option_pnl,
        "Futures_PnL_pts": futures_pnl,
        "Gross_PnL_pts": gross_pnl_points,
        "Hedge_Trades": hedge_trades,
        "Hedge_Cost_pts": hedge_cost_points,
        "Net_PnL_pts": net_pnl_points,
        "Net_PnL_rupees": net_pnl_rupees
    }


# ==========================================
# RUN OVER ALL TRADES
# ==========================================
results = []
for i, row in trades_raw.iterrows():
    res = gamma_scalp_for_trade(row)
    if res is not None:
        results.append(res)
    else:
        print(f"Skipped trade {i} (insufficient data or mismatch).")

if not results:
    print("\nNo valid gamma-scalping simulations completed.")
else:
    res_df = pd.DataFrame(results)
    print("\n========= GAMMA SCALPING SUMMARY =========")
    print(res_df[['Date', 'Entry_Time', 'Exit_Time', 'ATM_Strike',
                  'Net_PnL_pts', 'Net_PnL_rupees']].head())

    total_pts = res_df['Net_PnL_pts'].sum()
    total_rs  = res_df['Net_PnL_rupees'].sum()
    print("\nTOTAL Net Gamma PnL:")
    print(f"  {total_pts:.2f} points")
    print(f"  ₹{total_rs:,.2f}")

    print("\nPer-trade stats:")
    print(f"  Trades: {len(res_df)}")
    print(f"  Avg per trade: {res_df['Net_PnL_pts'].mean():.2f} pts "
          f"(₹{res_df['Net_PnL_rupees'].mean():.2f})")
    print(f"  Win rate: "
          f"{(res_df['Net_PnL_pts'] > 0).mean() * 100:.1f}%")

    # Save to CSV
    out_file = "gamma_scalping_results.csv"
    res_df.to_csv(out_file, index=False)
    print(f"\nDetailed gamma scalping results saved to: {out_file}")



--- year=2025/month=11/day=04/Futures/NIFTY/NIFTY25NOVFUT.parquet ---


DELTA-HEDGED GAMMA SCALPING

Loading futures data...
Futures ticks: 34,540
From 2025-11-04 08:46:29.948000 to 2025-11-04 15:28:32.829000

Loading kinetic trade windows...


FileNotFoundError: [Errno 2] No such file or directory: 'kinetic_full_backtest_trades.csv'

In [4]:
"""
AWS FUTURES SCALPER: HYPER-ADAPTIVE + TRAILING
===============================================
Logic: 
1. Core: RSI Kinetic Engine (51% Win Rate).
2. Defense: Side-Specific Kill Switch tightened to -1000 (Fail Fast).
3. Offense: Active Trailing Stop to lock profits.
   - +25 pts -> Move SL to Breakeven.
   - +50 pts -> Move SL to +25 pts.
"""

import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
YEAR = 2025
MONTH = 11
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25NOVFUT"

# Strategy Parameters
LOT_SIZE = 75
KINETIC_THRESHOLD = 37500
MAX_HOLD_SECONDS = 1200  # 20 mins

# RSI Filter Settings
RSI_PERIOD = 200     
RSI_OVERBOUGHT = 70.0
RSI_OVERSOLD = 30.0

# Trade Management
STOP_LOSS_POINTS = 30.0  
TAKE_PROFIT_POINTS = 80.0 # Extended TP to allow trailing

# Trailing Settings
BE_TRIGGER = 25.0       # Points profit to move SL to BE
TRAIL_TRIGGER = 50.0    # Points profit to start trailing
TRAIL_LOCK = 25.0       # Points locked when trailing starts

# Risk Management
SIDE_MAX_LOSS = -1000.0  # Tighter leash (approx 13 pts)
DAILY_MAX_LOSS = -3000.0 

# Costs
FUTURES_ROUND_TRIP_COST = 1.0

print(f"\n{'='*60}")
print(f"HYPER-ADAPTIVE SCALPER - {MONTH}/{YEAR}")
print(f"Logic: Kill Side < {SIDE_MAX_LOSS} | BE @ +{BE_TRIGGER} | Trail @ +{TRAIL_TRIGGER}")
print(f"{'='*60}\n")


# ==========================================
# S3 UTILITIES
# ==========================================
def list_s3(prefix: str):
    s3 = boto3.client("s3")
    keys = []
    continuation = None
    while True:
        kwargs = {"Bucket": BUCKET, "Prefix": prefix}
        if continuation:
            kwargs["ContinuationToken"] = continuation
        resp = s3.list_objects_v2(**kwargs)
        if "Contents" in resp:
            for obj in resp["Contents"]:
                keys.append(obj["Key"])
        if resp.get("IsTruncated"):
            continuation = resp.get("NextContinuationToken")
        else:
            break
    return keys

def download_parquet_to_path(key: str, local_path: str):
    if os.path.exists(local_path):
        return True
    s3 = boto3.client("s3")
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        with open(local_path, "wb") as f:
            f.write(obj["Body"].read())
        return True
    except Exception as e:
        return False


# ==========================================
# FAST TICK RSI
# ==========================================
class TickRSI:
    def __init__(self, period=200):
        self.period = period
        self.avg_gain = 0.0
        self.avg_loss = 0.0
        self.initialized = False
        self.count = 0

    def update(self, change):
        gain = max(0, change)
        loss = max(0, -change)
        
        if not self.initialized:
            self.avg_gain += gain
            self.avg_loss += loss
            self.count += 1
            if self.count >= self.period:
                self.avg_gain /= self.period
                self.avg_loss /= self.period
                self.initialized = True
        else:
            self.avg_gain = ((self.avg_gain * (self.period - 1)) + gain) / self.period
            self.avg_loss = ((self.avg_loss * (self.period - 1)) + loss) / self.period
            
    def get_rsi(self):
        if not self.initialized: return 50.0
        if self.avg_loss == 0: return 100.0
        rs = self.avg_gain / self.avg_loss
        return 100.0 - (100.0 / (1.0 + rs))


# ==========================================
# FAST DIRECTIONAL ENGINE
# ==========================================
class FastDirectionalEngine:
    def __init__(self, buffer_size=50):
        self.buffer_size = buffer_size
        self.price_buffer = deque(maxlen=buffer_size)
        self.volume_buffer = deque(maxlen=buffer_size)
        
    def add_tick(self, ltp, volume):
        self.price_buffer.append(ltp)
        self.volume_buffer.append(volume)
    
    def calculate_direction(self):
        if len(self.price_buffer) < self.buffer_size:
            return 0
        
        prices = np.array(self.price_buffer)
        vols = np.array(self.volume_buffer)
        
        vol_diff = np.diff(vols)
        vol_diff = np.where(vol_diff < 0, 0, vol_diff)
        
        if np.sum(vol_diff) == 0:
             return 1 if prices[-1] > prices[0] else 2
        
        trade_prices = prices[1:]
        vwap = np.average(trade_prices, weights=vol_diff)
        
        if prices[-1] > vwap: 
            return 1 # LONG
        else: 
            return 2 # SHORT


# ==========================================
# RSI KINETIC BRAIN
# ==========================================
class RSIKineticBrain:
    def __init__(self, threshold=37500):
        self.threshold = threshold
        self.ke_buffer = deque(maxlen=50)
        self.directional = FastDirectionalEngine(buffer_size=50)
        self.rsi = TickRSI(period=RSI_PERIOD)
        self.last_ltp = None
        self.last_score = 0
    
    def tick(self, ltp, volume):
        # 1. Update RSI
        if self.last_ltp is not None:
            change = ltp - self.last_ltp
            self.rsi.update(change)
        self.last_ltp = ltp
        
        # 2. Update Buffers
        self.ke_buffer.append((ltp, volume))
        self.directional.add_tick(ltp, volume)
        
        if len(self.ke_buffer) < 50:
            return 0
            
        # 3. Calculate Kinetic Energy
        arr = np.array(self.ke_buffer)
        prices_ke = arr[:, 0]
        vols_ke = arr[:, 1]
        
        v_diff = np.diff(vols_ke)
        v_diff = np.where(v_diff < 0, 0, v_diff)
        traded_vol = np.sum(v_diff)
        displacement = abs(prices_ke[-1] - prices_ke[0])
        
        ke_score = traded_vol / (displacement + 0.05)
        self.last_score = ke_score
        
        # 4. Check Signal
        if ke_score > self.threshold:
            raw_dir = self.directional.calculate_direction()
            current_rsi = self.rsi.get_rsi()
            
            # 5. RSI VETO
            if raw_dir == 1: 
                if current_rsi < RSI_OVERBOUGHT: return 1
            elif raw_dir == 2: 
                if current_rsi > RSI_OVERSOLD: return 2
            
        return 0


# ==========================================
# DATA LOADING
# ==========================================
def download_futures_for_day(year, month, day, symbol, fut_ts, day_folder):
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{symbol}/{fut_ts}.parquet"
    local_path = os.path.join(day_folder, "FUT.parquet")
    success = download_parquet_to_path(key, local_path)
    if not success: return None
    
    try:
        df = pd.read_parquet(local_path)
    except: return None

    df["DateTime"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str), dayfirst=True, errors="coerce")
    df["LTP"] = pd.to_numeric(df["LTP"], errors="coerce")
    df = df.dropna(subset=["DateTime", "LTP"]).sort_values("DateTime").reset_index(drop=True)
    
    if "Volume" in df.columns: df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
    elif "OpenInterest" in df.columns: df["Volume"] = pd.to_numeric(df["OpenInterest"], errors="coerce")
    else: df["Volume"] = 0.0
    
    return df

def process_day_trades(df, day_folder):
    brain = RSIKineticBrain(threshold=KINETIC_THRESHOLD)
    completed_trades = []
    
    # Active Trade State
    in_trade = False
    entry_time = None
    entry_price = 0.0
    side = None 
    highest_pnl = 0.0 # Track max floating profit
    
    # RISK STATE
    daily_pnl = 0.0
    long_pnl_today = 0.0
    short_pnl_today = 0.0
    
    for _, row in df.iterrows():
        if daily_pnl < DAILY_MAX_LOSS: break 
            
        ts = row["DateTime"]
        ltp = float(row["LTP"])
        vol = float(row["Volume"])
        
        # --- TRADE MANAGEMENT ---
        if in_trade:
            duration = (ts - entry_time).total_seconds()
            exit_signal = False
            exit_reason = ""
            
            # Calculate Floating PnL
            if side == "LONG":
                float_pnl = ltp - entry_price
            else:
                float_pnl = entry_price - ltp
                
            # Update Peak PnL for Trailing
            if float_pnl > highest_pnl:
                highest_pnl = float_pnl
            
            # --- DYNAMIC STOP LOSS LOGIC ---
            current_sl_pts = STOP_LOSS_POINTS # Default 30
            
            if highest_pnl >= TRAIL_TRIGGER: # Hit +50?
                current_sl_pts = -TRAIL_LOCK # Lock +25 (SL becomes Take Profit)
            elif highest_pnl >= BE_TRIGGER: # Hit +25?
                current_sl_pts = -1.0 # Breakeven (+1 for fees)
            
            # Check Exits
            if float_pnl <= -current_sl_pts: 
                exit_signal = True
                if current_sl_pts < 0: exit_reason = "TRAIL/BE"
                else: exit_reason = "SL"
            elif float_pnl >= TAKE_PROFIT_POINTS: 
                exit_signal = True; exit_reason = "TP"
            elif duration >= MAX_HOLD_SECONDS:
                exit_signal = True; exit_reason = "TIME"
                
            if exit_signal:
                gross_pnl = float_pnl * LOT_SIZE
                cost = FUTURES_ROUND_TRIP_COST * LOT_SIZE
                net_pnl = gross_pnl - cost
                
                daily_pnl += net_pnl
                if side == "LONG": long_pnl_today += net_pnl
                else: short_pnl_today += net_pnl
                
                completed_trades.append({
                    "Date": ts.date(),
                    "Entry_Time": entry_time,
                    "Side": side,
                    "Net_Pts": float_pnl - FUTURES_ROUND_TRIP_COST,
                    "Net_INR": net_pnl,
                    "Exit_Reason": exit_reason
                })
                in_trade = False
                highest_pnl = 0.0 # Reset
                continue 

        # --- SIGNAL GENERATION ---
        if not in_trade:
            sig = brain.tick(ltp, vol)
            
            if sig == 1: 
                if long_pnl_today > SIDE_MAX_LOSS: 
                    in_trade = True; entry_time = ts; entry_price = ltp; side = "LONG"
            elif sig == 2: 
                if short_pnl_today > SIDE_MAX_LOSS: 
                    in_trade = True; entry_time = ts; entry_price = ltp; side = "SHORT"
        else:
            brain.tick(ltp, vol)

    trades_df = pd.DataFrame(completed_trades)
    return trades_df


# ==========================================
# MAIN EXECUTION
# ==========================================
def main():
    all_trades = []
    
    for day in range(1, 31):
        day_folder = f"data/{YEAR}-{MONTH:02d}-{day:02d}/"
        os.makedirs(day_folder, exist_ok=True)
        
        print(f"\n{day:02d}/{MONTH}/{YEAR}...", end=" ")
        
        df = download_futures_for_day(YEAR, MONTH, day, SYMBOL, FUT_TS, day_folder)
        
        if df is None or df.empty:
            print("No data")
            continue
        
        trades_df = process_day_trades(df, day_folder)
        
        if not trades_df.empty:
            daily_csv = os.path.join(day_folder, "futures_results.csv")
            trades_df.to_csv(daily_csv, index=False)
            
            pnl = trades_df['Net_INR'].sum()
            n_trades = len(trades_df)
            sl_hits = len(trades_df[trades_df['Exit_Reason'] == 'SL'])
            be_hits = len(trades_df[trades_df['Exit_Reason'] == 'TRAIL/BE'])
            
            l_pnl = trades_df[trades_df['Side']=='LONG']['Net_INR'].sum()
            s_pnl = trades_df[trades_df['Side']=='SHORT']['Net_INR'].sum()
            
            print(f"Trades: {n_trades} (SL:{sl_hits}/BE:{be_hits}) | PnL: ₹{pnl:,.0f} [L:{l_pnl:.0f} S:{s_pnl:.0f}]")
            if pnl < DAILY_MAX_LOSS: print(f"   [RISK] Global Stop Hit!")
            
            all_trades.append(trades_df)
        else:
            print("No trades")
    
    if all_trades:
        full_df = pd.concat(all_trades, ignore_index=True)
        
        total_pnl = full_df['Net_INR'].sum()
        win_rate = (full_df['Net_Pts'] > 0).mean() * 100
        
        longs = full_df[full_df['Side'] == 'LONG']
        shorts = full_df[full_df['Side'] == 'SHORT']
        
        print("\n" + "="*60)
        print(f"HYPER-ADAPTIVE SUMMARY: {MONTH}/{YEAR}")
        print("="*60)
        print(f"Total Net PnL:      ₹{total_pnl:,.2f}")
        print(f"Win Rate:           {win_rate:.1f}%")
        print(f"Total Trades:       {len(full_df)}")
        print(f"  - Longs:          {len(longs)} (₹{longs['Net_INR'].sum():,.0f})")
        print(f"  - Shorts:         {len(shorts)} (₹{shorts['Net_INR'].sum():,.0f})")
        
        final_csv = f"data/nov_{YEAR}_hyper_results.csv"
        full_df.to_csv(final_csv, index=False)
        print(f"\nResults saved: {final_csv}")
    else:
        print("\nNo trades for the month.")

if __name__ == "__main__":
    main()


HYPER-ADAPTIVE SCALPER - 11/2025
Logic: Kill Side < -1000.0 | BE @ +25.0 | Trail @ +50.0


01/11/2025... No data

02/11/2025... No data

03/11/2025... No data

04/11/2025... Trades: 19 (SL:1/BE:1) | PnL: ₹7,793 [L:-2557 S:10350]

05/11/2025... No data

06/11/2025... Trades: 3 (SL:2/BE:1) | PnL: ₹-4,868 [L:-2543 S:-2325]
   [RISK] Global Stop Hit!

07/11/2025... Trades: 3 (SL:2/BE:0) | PnL: ₹-3,743 [L:-2325 S:-1418]
   [RISK] Global Stop Hit!

08/11/2025... No data

09/11/2025... No data

10/11/2025... No data

11/11/2025... Trades: 15 (SL:6/BE:1) | PnL: ₹-3,547 [L:-2325 S:-1222]
   [RISK] Global Stop Hit!

12/11/2025... Trades: 2 (SL:2/BE:0) | PnL: ₹-4,695 [L:-2370 S:-2325]
   [RISK] Global Stop Hit!

13/11/2025... Trades: 21 (SL:6/BE:0) | PnL: ₹-1,545 [L:-1088 S:-458]

14/11/2025... No data

15/11/2025... No data

16/11/2025... No data

17/11/2025... Trades: 2 (SL:2/BE:0) | PnL: ₹-5,010 [L:-2460 S:-2550]
   [RISK] Global Stop Hit!

18/11/2025... Trades: 2 (SL:2/BE:0) | PnL: ₹-4,703 [

In [21]:
"""
AWS FUTURES SCALPER: KINETIC DIRECTIONAL BRAIN v2
=================================================
Logic:
- Detect big Kinetic bursts (volume / displacement)
- Only trade when:
    * Strong Kinetic burst
    * Higher-timeframe trend (regime) is clearly UP or DOWN
    * Signed orderflow agrees with that regime
    * Local range position confirms (breakout / breakdown zone)
"""

import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"

YEAR = 2025
MONTH = 11
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25NOVFUT"  # change if contract differs

LOT_SIZE = 75

# --- Kinetic / Direction Params ---
KINETIC_THRESHOLD = 37500
PRICE_WINDOW = 40         # for local range
VOL_WINDOW = 40           # for KE + signed flow
FAST_LOOKBACK = 10        # short slope
SLOW_LOOKBACK = 30        # medium slope
REGIME_WINDOW = 200       # higher-timeframe trend window
MIN_VOTES = 2             # minimum directional votes

SIGNED_VOL_THRESH = 0.25  # stricter orderflow bias
REGIME_SLOPE_THRESH = 1e-5  # normalized slope threshold

# --- Trade Management ---
MAX_HOLD_SECONDS = 900
STOP_LOSS_POINTS = 25.0
TAKE_PROFIT_POINTS = 50.0

# --- Costs & Risk ---
FUTURES_ROUND_TRIP_COST = 1.0   # pts per round trip
DAILY_MAX_LOSS = -3000.0        # INR

print(f"\n{'='*60}")
print(f"KINETIC DIRECTIONAL FUTURES SCALPER v2 - {MONTH}/{YEAR}")
print(f"Symbol: {SYMBOL}, Contract: {FUT_TS}")
print(f"{'='*60}\n")


# ==========================================
# S3 UTILITIES
# ==========================================
def list_s3(prefix: str):
    s3 = boto3.client("s3")
    keys = []
    continuation = None
    while True:
        kwargs = {"Bucket": BUCKET, "Prefix": prefix}
        if continuation:
            kwargs["ContinuationToken"] = continuation
        resp = s3.list_objects_v2(**kwargs)
        if "Contents" in resp:
            for obj in resp["Contents"]:
                keys.append(obj["Key"])
        if resp.get("IsTruncated"):
            continuation = resp.get("NextContinuationToken")
        else:
            break
    return keys


def download_parquet_to_path(key: str, local_path: str):
    if os.path.exists(local_path):
        return True
    s3 = boto3.client("s3")
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        with open(local_path, "wb") as f:
            f.write(obj["Body"].read())
        return True
    except Exception:
        return False


# ==========================================
# KINETIC DIRECTIONAL BRAIN v2
# ==========================================
class KineticDirectionalBrain:
    """
    tick(ltp, volume) ->
        0  = no signal
        +1 = LONG
        -1 = SHORT
    """

    def __init__(
        self,
        kinetic_threshold=KINETIC_THRESHOLD,
        price_window=PRICE_WINDOW,
        vol_window=VOL_WINDOW,
        fast_lookback=FAST_LOOKBACK,
        slow_lookback=SLOW_LOOKBACK,
        regime_window=REGIME_WINDOW,
        min_votes=MIN_VOTES,
        signed_vol_thresh=SIGNED_VOL_THRESH,
        regime_slope_thresh=REGIME_SLOPE_THRESH,
    ):
        self.kinetic_threshold = kinetic_threshold
        self.price_window = price_window
        self.vol_window = vol_window
        self.fast_lookback = fast_lookback
        self.slow_lookback = slow_lookback
        self.regime_window = regime_window
        self.min_votes = min_votes
        self.signed_vol_thresh = signed_vol_thresh
        self.regime_slope_thresh = regime_slope_thresh

        self.price_buf = deque(
            maxlen=max(price_window, vol_window, regime_window)
        )
        self.vol_buf = deque(maxlen=vol_window)
        self.last_ke_score = 0.0

    def _lin_slope(self, arr):
        n = len(arr)
        if n < 2:
            return 0.0
        x = np.arange(n, dtype=float)
        y = np.asarray(arr, dtype=float)
        x_mean = x.mean()
        y_mean = y.mean()
        num = np.sum((x - x_mean) * (y - y_mean))
        den = np.sum((x - x_mean) ** 2) + 1e-9
        return num / den

    def tick(self, ltp, volume):
        ltp = float(ltp)
        volume = float(volume)

        self.price_buf.append(ltp)
        self.vol_buf.append(volume)

        if (
            len(self.vol_buf) < self.vol_window
            or len(self.price_buf) < self.price_window
            or len(self.price_buf) < self.regime_window
        ):
            return 0

        prices = np.asarray(self.price_buf, dtype=float)
        vols = np.asarray(self.vol_buf, dtype=float)

        # ---------- 1) KINETIC SCORE ----------
        vol_diff = np.diff(vols)
        trade_vol = np.where(vol_diff > 0, vol_diff, 0.0)
        if trade_vol.sum() <= 0:
            return 0

        displacement = abs(prices[-1] - prices[-self.vol_window])
        ke_score = trade_vol.sum() / (displacement + 0.05)
        self.last_ke_score = ke_score

        if ke_score < self.kinetic_threshold:
            return 0

        # ---------- 2) SIGNED ORDERFLOW ----------
        price_short = prices[-self.vol_window:]
        dp = np.diff(price_short)
        v = trade_vol[-len(dp):]
        signed_vol = np.sum(np.sign(dp) * v)
        total_trade_vol = v.sum() + 1e-9
        signed_vol_ratio = signed_vol / total_trade_vol   # -1..+1

        # If orderflow is not clearly biased, ignore
        if abs(signed_vol_ratio) < self.signed_vol_thresh:
            return 0

        # ---------- 3) SLOPES ----------
        fast_len = min(self.fast_lookback, len(prices))
        slow_len = min(self.slow_lookback, len(prices))
        regime_len = min(self.regime_window, len(prices))

        fast_slice = prices[-fast_len:]
        slow_slice = prices[-slow_len:]
        regime_slice = prices[-regime_len:]

        fast_slope = self._lin_slope(fast_slice)
        slow_slope = self._lin_slope(slow_slice)
        regime_slope = self._lin_slope(regime_slice)

        price_scale = np.mean(regime_slice) + 1e-9
        fast_slope_norm = fast_slope / price_scale
        slow_slope_norm = slow_slope / price_scale
        regime_slope_norm = regime_slope / price_scale

        # ---------- 4) REGIME FILTER ----------
        if regime_slope_norm > self.regime_slope_thresh:
            regime = "UP"
        elif regime_slope_norm < -self.regime_slope_thresh:
            regime = "DOWN"
        else:
            return 0  # FLAT regime -> do nothing

        # ---------- 5) RANGE POSITION ----------
        window_slice = prices[-self.price_window:]
        p_min = window_slice.min()
        p_max = window_slice.max()
        rng = (p_max - p_min) + 1e-9
        range_pos = (window_slice[-1] - p_min) / rng  # 0 = low, 1 = high

        # ---------- 6) VOTES ----------
        long_votes = 0
        short_votes = 0

        # (a) Signed orderflow
        if signed_vol_ratio > 0:
            long_votes += 1
        else:
            short_votes += 1

        # (b) Fast slope
        if fast_slope_norm > 0:
            long_votes += 1
        elif fast_slope_norm < 0:
            short_votes += 1

        # (c) Slow slope
        if slow_slope_norm > 0:
            long_votes += 1
        elif slow_slope_norm < 0:
            short_votes += 1

        # Base direction from votes
        if long_votes >= short_votes + 1:
            direction = +1
            votes = long_votes
        elif short_votes >= long_votes + 1:
            direction = -1
            votes = short_votes
        else:
            return 0

        if votes < self.min_votes:
            return 0

        # ---------- 7) ALIGN WITH REGIME ----------
        if regime == "UP" and direction < 0:
            return 0
        if regime == "DOWN" and direction > 0:
            return 0

        # ---------- 8) RANGE CONFIRMATION ----------
        if regime == "UP":
            if range_pos < 0.5:
                return 0
        else:
            if range_pos > 0.5:
                return 0

        return direction


# ==========================================
# DATA LOADING
# ==========================================
def download_futures_for_day(year, month, day, symbol, fut_ts, day_folder):
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{symbol}/{fut_ts}.parquet"
    local_path = os.path.join(day_folder, "FUT.parquet")
    ok = download_parquet_to_path(key, local_path)
    if not ok:
        return None

    try:
        df = pd.read_parquet(local_path)
    except Exception:
        return None

    if "DateTime" not in df.columns:
        if "Date" in df.columns and "Time" in df.columns:
            df["DateTime"] = pd.to_datetime(
                df["Date"].astype(str) + " " + df["Time"].astype(str),
                dayfirst=True,
                errors="coerce",
            )
        else:
            return None

    df = df.dropna(subset=["DateTime"])
    df["LTP"] = pd.to_numeric(df.get("LTP"), errors="coerce")
    df = df.dropna(subset=["LTP"])

    if "Volume" in df.columns:
        df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
    elif "OpenInterest" in df.columns:
        df["Volume"] = pd.to_numeric(df["OpenInterest"], errors="coerce")
    else:
        df["Volume"] = 0.0

    df = df.sort_values("DateTime").reset_index(drop=True)

    df = df[
        (df["DateTime"].dt.time >= datetime.time(9, 15))
        & (df["DateTime"].dt.time <= datetime.time(15, 15))
    ].reset_index(drop=True)

    return df


# ==========================================
# DAY PROCESSING
# ==========================================
def process_day_trades(df):
    brain = KineticDirectionalBrain()

    trades = []
    in_trade = False
    entry_time = None
    entry_price = 0.0
    side = None
    daily_pnl = 0.0

    for _, row in df.iterrows():
        ts = row["DateTime"]
        ltp = float(row["LTP"])
        vol = float(row["Volume"])

        if daily_pnl <= DAILY_MAX_LOSS:
            break

        if in_trade:
            duration = (ts - entry_time).total_seconds()
            exit_signal = False
            exit_reason = ""
            pnl_pts = 0.0

            if side == "LONG":
                pnl_pts = ltp - entry_price
                if pnl_pts <= -STOP_LOSS_POINTS:
                    exit_signal = True
                    exit_reason = "SL"
                elif pnl_pts >= TAKE_PROFIT_POINTS:
                    exit_signal = True
                    exit_reason = "TP"
            else:
                pnl_pts = entry_price - ltp
                if pnl_pts <= -STOP_LOSS_POINTS:
                    exit_signal = True
                    exit_reason = "SL"
                elif pnl_pts >= TAKE_PROFIT_POINTS:
                    exit_signal = True
                    exit_reason = "TP"

            if not exit_signal and duration >= MAX_HOLD_SECONDS:
                exit_signal = True
                exit_reason = "TIME"

            if exit_signal:
                gross_pnl_inr = pnl_pts * LOT_SIZE
                cost_inr = FUTURES_ROUND_TRIP_COST * LOT_SIZE
                net_inr = gross_pnl_inr - cost_inr
                net_pts_after_cost = pnl_pts - FUTURES_ROUND_TRIP_COST
                daily_pnl += net_inr

                trades.append({
                    "Date": ts.date(),
                    "Entry_Time": entry_time,
                    "Exit_Time": ts,
                    "Side": side,
                    "Entry_Price": entry_price,
                    "Exit_Price": ltp,
                    "Raw_PnL_Pts": pnl_pts,
                    "Net_Pts": net_pts_after_cost,
                    "Net_INR": net_inr,
                    "Exit_Reason": exit_reason,
                })

                in_trade = False
                side = None
                entry_time = None
                entry_price = 0.0
                continue

        if not in_trade:
            sig = brain.tick(ltp, vol)
            if sig == 1:
                in_trade = True
                side = "LONG"
                entry_time = ts
                entry_price = ltp
            elif sig == -1:
                in_trade = True
                side = "SHORT"
                entry_time = ts
                entry_price = ltp
        else:
            brain.tick(ltp, vol)

    return pd.DataFrame(trades)


# ==========================================
# MAIN
# ==========================================
def main():
    all_trades = []

    for day in range(1, 31):
        day_folder = f"data/{YEAR}-{MONTH:02d}-{day:02d}/"
        os.makedirs(day_folder, exist_ok=True)

        print(f"{day:02d}/{MONTH}/{YEAR}...", end=" ")

        df = download_futures_for_day(YEAR, MONTH, day, SYMBOL, FUT_TS, day_folder)
        if df is None or df.empty:
            print("No data")
            continue

        trades_df = process_day_trades(df)

        if trades_df.empty:
            print("No trades")
            continue

        day_pnl = trades_df["Net_INR"].sum()
        n_trades = len(trades_df)
        sl_hits = (trades_df["Exit_Reason"] == "SL").sum()
        tp_hits = (trades_df["Exit_Reason"] == "TP").sum()

        print(f"Trades: {n_trades} (SL:{sl_hits}/TP:{tp_hits}) | PnL: ₹{day_pnl:,.0f}")
        if day_pnl <= DAILY_MAX_LOSS:
            print("   [RISK] Daily Stop Loss Breached!")

        daily_csv = os.path.join(day_folder, "futures_directional_v2_results.csv")
        trades_df.to_csv(daily_csv, index=False)
        all_trades.append(trades_df)

    if not all_trades:
        print("\nNo trades for the month.")
        return

    full_df = pd.concat(all_trades, ignore_index=True)
    total_pnl = full_df["Net_INR"].sum()
    win_rate = (full_df["Net_Pts"] > 0).mean() * 100
    longs = full_df[full_df["Side"] == "LONG"]
    shorts = full_df[full_df["Side"] == "SHORT"]

    print("\n" + "="*60)
    print(f"KINETIC DIRECTIONAL SUMMARY v2: {MONTH}/{YEAR}")
    print("="*60)
    print(f"Total Net PnL:      ₹{total_pnl:,.2f}")
    print(f"Win Rate:           {win_rate:.1f}%")
    print(f"Total Trades:       {len(full_df)}")
    print(f"  - Longs:          {len(longs)} (₹{longs['Net_INR'].sum():,.0f})")
    print(f"  - Shorts:         {len(shorts)} (₹{shorts['Net_INR'].sum():,.0f})")

    out_csv = f"data/nov_{YEAR}_kinetic_directional_v2_results.csv"
    full_df.to_csv(out_csv, index=False)
    print(f"\nResults saved: {out_csv}")


if __name__ == "__main__":
    main()



KINETIC DIRECTIONAL FUTURES SCALPER v2 - 11/2025
Symbol: NIFTY, Contract: NIFTY25NOVFUT

01/11/2025... No data
02/11/2025... No data
03/11/2025... No data
04/11/2025... No trades
05/11/2025... No data
06/11/2025... No trades
07/11/2025... Trades: 1 (SL:0/TP:0) | PnL: ₹-53
08/11/2025... No data
09/11/2025... No data
10/11/2025... No data
11/11/2025... No trades
12/11/2025... No trades
13/11/2025... No trades
14/11/2025... No data
15/11/2025... No data
16/11/2025... No data
17/11/2025... No trades
18/11/2025... No trades
19/11/2025... No trades
20/11/2025... No trades
21/11/2025... No trades
22/11/2025... No data
23/11/2025... No data
24/11/2025... No trades
25/11/2025... No trades
26/11/2025... No data
27/11/2025... No data
28/11/2025... No data
29/11/2025... No data
30/11/2025... No data

KINETIC DIRECTIONAL SUMMARY v2: 11/2025
Total Net PnL:      ₹-52.50
Win Rate:           0.0%
Total Trades:       1
  - Longs:          1 (₹-53)
  - Shorts:         0 (₹0)

Results saved: data/nov_202